In [ ]:
label = ['cat' , 'dog']
def xml (file):
    tree = ET.parse(file)
    root = tree.getroot()
    
    for obje in root.findall('object'):
        name = obje.find('name').text
        
        label1 = label.index(name)
        
        bndbox = obje.find('bndbox')
        
        xmin  = bndbox.find('xmin').text
        ymin  = bndbox.find('ymin').text
        xmax  = bndbox.find('xmax').text
        ymax  = bndbox.find('ymax').text
        
        return([int(label1) , int(xmin) ,int(ymin) , int(xmax) , int(ymax)])
    
    
    

In [ ]:
path = '/kaggle/input/dog-and-cat-detection/annotations/'
# Sorted se hamari file jese data set me h order me use order me dikhengi
text_file = ['/kaggle/input/dog-and-cat-detection/annotations/' + f for f in sorted(os.listdir(path))]
y = []


for file in range(1500):
    y.append(xml(text_file[file]))
    

In [ ]:

y = np.array(y)
print(type(y))


In [ ]:
path = '/kaggle/input/dog-and-cat-detection/images/'
img_dir = os.path.join(path,"*g")
files = glob.glob(img_dir)
files.sort()

In [ ]:
import tensorflow as tf
import pandas as pd
import os
import numpy as np
from xml.dom import minidom




import cv2
import matplotlib.pyplot as plt
import glob
from sklearn.model_selection import train_test_split
from skimage import io

In [ ]:
IMAGE_SIZE = 224
image_folder_path = "/kaggle/input/dog-and-cat-detection/images"

img_data = []

for img in range(1500):
    img_path = os.path.join(image_folder_path,sorted(os.listdir(image_folder_path))[img])
    img = cv2.imread(img_path)
    img = cv2.resize(img,(IMAGE_SIZE , IMAGE_SIZE))
    img_data.append(np.array(img))


In [ ]:
X = np.array(img_data)


In [ ]:
print(type(X))
print(type(y))

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)
X_train.shape,y_train.shape,X_test.shape,y_test.shape

In [ ]:
y1_train = y_train[:,0:1]
y1_test = y_test[:,0:1]
y2_train = y_train[:,1:]
y2_test = y_test[:,1:]
y1_train.shape,y1_test.shape,y2_train.shape,y2_test.shape

In [ ]:
# Yha hmne class ko one hot encoder kiya h 
y1_train = tf.keras.utils.to_categorical(y1_train)
y1_test = tf.keras.utils.to_categorical(y1_test)


In [ ]:
inp = tf.keras.layers.Input(shape=(224,224,3))
res = tf.keras.applications.ResNet152V2(include_top=False,weights="imagenet" ) (inp)
flatten = tf.keras.layers.Flatten() (res)
d1 = tf.keras.layers.Dense(128,activation='relu') (flatten)

d2 = tf.keras.layers.Dense(128,activation='relu') (d1)
#yha ye 1 or two hmara class ke index ke uper depent h or hmesha normilized label ko nhi kiya jata h bndbox ke 
out1 = tf.keras.layers.Dense(2,activation='softmax') (d2)

d3 = tf.keras.layers.Dense(256,activation='relu') (d2)
d4 = tf.keras.layers.Dense(256,activation='relu') (d3)

out2 = tf.keras.layers.Dense(4) (d4)

model = tf.keras.models.Model(inputs=inp,outputs=[out1,out2])
model.summary()

In [ ]:
tf.keras.utils.plot_model(model)

In [ ]:
model.compile(optimizer='adam',loss=['categorical_crossentropy','huber'])

In [ ]:
model.fit(X_train,[y1_train,y2_train],epochs=80)

# TESTING NEW

In [ ]:
pred_img = cv2.imread('/kaggle/input/dog-and-cat-detection/images/Cats_Test0.png')
pred_img = cv2.resize(pred_img,(224, 224))
# pred_img = pred_img.reshape(1,224,224,3)


c,bnd = model.predict(np.asarray([pred_img]))

In [ ]:
# 1752 1541 6138 5508

bnd = np.round(bnd)
bnd = np.array(bnd.astype('int64'))
xmin , ymin , xmax , ymax      =    bnd[0][0] , bnd[0][1] ,bnd[0][2] ,bnd[0][3] 
print(xmin , ymin , xmax , ymax )

In [ ]:
img = cv2.imread('/kaggle/input/dog-and-cat-detection/images/Cats_Test0.png',cv2.COLOR_BGR2RGB)
bnd_img=cv2.rectangle(img ,(xmin,ymin),(xmax,ymax),(255,0,0),2) 
plt.imshow(bnd_img)